In [ ]:
# comment out to run on drive + colab

# ! pip install transformers_interpret

In [ ]:
# from google.colab import drive
#
# drive.mount('/content/drive/')

In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import MultiLabelClassificationExplainer
import torch
from transformers import pipeline
from datasets import load_from_disk

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# comment out the filepath to run on drive + colab
filepath = "../models/scenario_level/conduction_control"
# filepath = "/content/drive/Shareddrives/AphasiaProject/models/scenario_level/conduction_control"

model = AutoModelForSequenceClassification.from_pretrained(filepath+"/model")
tokenizer = AutoTokenizer.from_pretrained(filepath+"/tokenizer", padding=True,
                                          truncation=True, return_tensors="pt",
                                          add_special_tokens=True, max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    model.to(device)
pipe = pipeline("text-classification", model=filepath+"/model", tokenizer=filepath+"/tokenizer", device=device, truncation=True, padding=True)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.config

DistilBertConfig {
  "_name_or_path": "../models/conduction_control/model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "CONTROL",
    "1": "CONDUCTION"
  },
  "initializer_range": 0.02,
  "label2id": {
    "CONDUCTION": 1,
    "CONTROL": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.39.0.dev0",
  "vocab_size": 30524
}

In [ ]:
model.config.id2label

{0: 'CONTROL', 1: 'CONDUCTION'}

In [ ]:
def interpret(data):
    for text, label in zip(list(data["preprocessed_text"]), list(data["label"])):
        if device == torch.device("cuda"):
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512).input_ids.to(device)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")
        else:
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(**inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")

In [ ]:
scenarios = ["Speech", "Important_Event", "Cinderella", "Stroke", "Cat"]
# make sure not to interpret on trained scenarios
texts = load_from_disk(filepath + "/dataset")["test"].remove_columns(["Unnamed: 0", "input_ids", "attention_mask"]).to_pandas()
texts["label"] = [model.config.id2label[x] for x in texts["label"]]
n = 3

# Speech scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[0])].sample(n=n)])

In [ ]:
interpret(data)

True:  CONDUCTION Pred:  CONDUCTION
Text:  it's doing not very much but it's okay.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.42),CONTROL,-0.65,[CLS] it ' s doing not very much but it ' s okay . [SEP]
n/a,(0.56),CONDUCTION,0.45,[CLS] it ' s doing not very much but it ' s okay . [SEP]


----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  and it's dark, no. sunny times really help. yeah. you believe it? close the doors. no close it. i every all the nights bad. if everything open and sunny i'm fine. i'm not good but a lot better. then i'm somewhat better. but i'm never very good. so not to her!


----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  how do they how do they what? okay. okay. okay. [FP] bad. bad. [FP] my [FP] me that i've been there then before that the stroke. [FP] it's it's [FP] deep. [FP] it doesn't it doesn't me at all. and it looks like [FP] echo hello echo. it doesn't like i'm saying it. but it's going like [FP] it doesn't somebody else. it really does.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  but [FP] at my age i very often [FP] forget the word that i want to say. and it's getting a little worse. names especially.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  everybody tells me i'm very articulate. just like a smattering of spanish, just really a few words.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  okay i speak good english. may i speak indian with her? okay.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.84),CONTROL,1.78,[CLS] okay i speak good english . may i speak indian with her ? okay . [SEP]
n/a,(0.15),CONDUCTION,-1.74,[CLS] okay i speak good english . may i speak indian with her ? okay . [SEP]


----------------------------------------------------------------------


# Important event scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[1])].sample(n=n)])

In [ ]:
interpret(data)

True:  CONDUCTION Pred:  CONDUCTION
Text:  okay. [UP] well well my sister had for it's the the stroke. something's wrong. it's almost like it's almost but it's it's gonna be that [FP] word. it's called [FP] suicide. a lot of i i just feel a lot of people they think when something's wrong. so and it's really it's going down. but it just happens. it's going almost the speech is almost gone. i understand. i i don't i i couldn't do it. so but then it's going better up. right now it's still on. it's still on right now.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  oh i think well actually meeting you at [FP] at at [FP] firstnamep and firstnamey's. and you remember you started you you know you were starting to do some of the work with me. and you had like pictures. and i hadta [FP] [FP] try to say what the words were and stuff like that. yeah i remember that. i was remembering that. and i was and i remembered that i was having a hard time to say what the words were. and [FP] you know and you in the beginning it was pretty easy. and then of course the words got more difficult. and [FP] so i mean that was a lot of the work that you did. and i think the reason why [FP] you were involved that you were involved with that is that [FP] we were worried that we couldn't even find a speech therapist. and i think that's how firstnamey found you you know? and then we finally then we then we did find someone to go down to athol. i hadta get better

----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  [FP] it was a long haul. it was a lot you know a lot of discussion with my wife eileen about possibly doing it. and we probably over the course of years we talked about maybe adopting maybe becoming foster parents. and [FP] at one point i would be kind of more wanting to do it. and eileen would be busy. her parents were ill. and we were trying to take care of them. and then it would swap. i was busy at work. and she was more interested in talking about it. it went back and forth for probably seven or eight years before kind of we were on the same page at the same time. and [FP] we made this decision to to that we should look possibly look at foster care. and we looked into it closely. and it just wasn't going to work right for us. and then we decided to to adopt. and [FP] we identified this little girl who was three years old and needed a family. and we [FP] looked into all of he

----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  and my friend who was having the same disease, he went to secondschoolname. and he did treatment similarly with the backbone surgery, herniated disc. and he was feeling not good even after the operation. make my case when i was treated by the doctor in medicalschoolname [FP] is a long time ago. and i never feel back of that same pain. it's completely recovered. the guy was my friend was complaining. so compared to that, i feel that [FP] it was successfully done. and i hadta go for fitness. i did the fitness. and then it get me full recovery. so that's a good news.


----------------------------------------------------------------------


# Cinderella scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[2])].sample(n=n)])

In [ ]:
interpret(data)

True:  CONDUCTION Pred:  CONDUCTION
Text:  no. it's very the book is hard hard. but i know the story. long time ago cinderella was cinderella cinderella was getting to the theater. go away because the time is eight the one o'clock. you? you? you? it's cinderella. cinderella. oh boy! teo the the two get married. and that was the story.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONTROL
Text:  and this little. yes. and i look forward years and years ago when you all loved it. when you loved you were younger and everything. this sort of thing read this when you were young believe me. and it was fun [FP] to show the children and as they get older. and the mother is in the charge, just comes around. you can see her. and there were some girls there were several girls they've had. and some were exciting and not approve. the other boy the other girl was very pleasant. it so pretty. and they are thinking of her as they get older. gonna get married for that one girl. and they met a man. no but it really after while suddenly he notice it not for him. and then eventually the woman met another man that he liked very well. and he became the president present got married. and [FP] to see them. but it was fun for the women when the children goer older. and then they got set. and 

----------------------------------------------------------------------


# Stroke scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    df = texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])]
    # no controls in stroke scenario
    if len(df) >= n:
        data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])].sample(n=n)])

In [ ]:
interpret(data)

True:  CONDUCTION Pred:  CONDUCTION
Text:  sure. i was well happened to me asleep at night. no pie nothing. i woke up at seven o'clock which i always do. and i go out with the dog. and my husband suddenly woke up himself. and he said. for goodness sake what are you saying? and i said. i'm talking. he said. your can't say anything. from then on i couldn't say anything. i could talking but it didn't say anything. so that's the way i spent from then on. i had the stroke. and it was closed here. and that's why i got that they hadta open it up. but i didn't talk well. well i'm fine. it happened across from both sides. and [FP] i hadta start [FP] talking. that was the thing. because i feel fine. high blood pressure but i've had it from the time i was thirty years old. and [FP] i work at physician. and my husband was a physician also. and [FP] my children were younger. much older for me. and so that's what's been going for me for working from then on to try and talk. i yeah well i just must d

----------------------------------------------------------------------


# Cat scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[4])].sample(n=n)])

In [ ]:
interpret(data)

True:  CONDUCTION Pred:  CONDUCTION
Text:  [FP] okay? oh. well that's that's [FP] that says it all. [FP] [FP] the tree is [FP] the big tree. and he's got [FP] a man is up i guess. i don't know. with the dog i think. with [FP] he's [FP] the dog is barking. i hate that. [FP]? not a in the care in the world. but [FP] the policeman show. well the fire fireman. the ladder. [FP] it's actually two men. and i guess the man is get down the [FP] the ladder. and this is the the girl is doing up with the cat. and the the bicycle, the trike. he's he is got that she has got that. and the ladder, a different ladder. see it's the picture's better.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  so the little girl is flying. because her trike because the cat is up there. and the up there. but somehow the ladder fell down. oh i guess he was long enough. and they're come with a higher lander. and he says. so it was all good. and the cat will is okay with the bird. but even though she's gonna sing sing. anyway sorry. okay i'm sorry. do what is lesser.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  oh okay. okay this is kind of a fun story. this is like the [FP] so [FP] there's a there's like a little girl. and she liked liked the the cat. there's also a dog. now the dog could belong belong to [FP] to the the their family perhaps. or it could be [FP] somebody else's [FP] neighbor. and they had a dog. and the dog actually scared was actually afraid of the [FP] the cat was afraid of the dog. and the dog ran up the tree. and it was [FP] on the up there pretty high. so his his father his father said. oh i'll get the cat. and i'll get a ladder. and i'll put the ladder up here. and i'll try to get the cat. so i got up there. and what happened though? the ladder fell down. and the little girl probably was driving on this little tight type. i don't know what you call that it starts with a k. but i don't think that that that has anything involved with the cat. i don't think th

----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  [FP] he himself though ends up being stuck. his facial expression indicates that he he's having difficulty moving one way or the other. and [FP] being of no help, the dog is [FP] barking, making a racket. and someone evidently called the fire department. and there are two fireman have rushed over to the tree with a ladder. and they're going to try to get him down. his ladder [FP] father's ladder [FP] is on the ground [FP] i guess too far for anyone to retrieve it for him. so the fire fire department has come to rescue him and the cat, i hope.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  so i guess the little girl has a dog and a cat maybe. and the dog chased the cat up into the tree. and then it looks like maybe she and dad tried to get the cat down. and then dad got stuck in the tree. so they hadta call for backup to come help the whole situation out.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  [FP] he was next to his neighbor's house when his neighbor's dog chased him up a tree [FP] because he was deathly terrified of dogs. at the same time, their neighbor's kid's cat also climbed up in the tree. and she was all freaked out about that. and the dog is barking. and someone called the fire department. and. i can't remember his name. tom was just besides himself with embarrassment because he's up a tree. and the fire department's there. and he never lived it down.


----------------------------------------------------------------------
